In [4]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

import tensorflow as tf
from sklearn import preprocessing
import numpy as np
import random
import matplotlib.pyplot as plt
import sys
from utils.CLR import*
from utils.LoadData import load_CW_Source
from Model.CNN import cnn_classifier
from sklearn.utils import shuffle

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
def train_model(X_profiling, Y_profiling, X_test, Y_test, model, save_file_name, epochs=150, batch_size=100,
                max_lr=1e-3):

    # Save model every epoch
    save_model = ModelCheckpoint(
                    filepath=save_file_name,
                    monitor='val_loss',
                    mode='min',
                    save_best_only=True)

    # Get the input layer shape
    input_layer_shape = model.get_layer(index=0).input_shape[0]

    # Sanity check
    if input_layer_shape[1] != len(X_profiling[0]):
        print("Error: model input shape %d instead of %d is not expected ..." % (
        input_layer_shape[1], len(X_profiling[0])))
        sys.exit(-1)
    Reshaped_X_profiling, Reshaped_X_test = X_profiling.reshape(
        (X_profiling.shape[0], X_profiling.shape[1], 1)), X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # One Cycle Policy
    lr_manager = OneCycleLR(max_lr=max_lr, end_percentage=0.2, scale_percentage=0.1, maximum_momentum=None,
                            minimum_momentum=None, verbose=True)

    callbacks = [save_model, lr_manager]

    history = model.fit(x=Reshaped_X_profiling, y=to_categorical(Y_profiling, num_classes=256),
                        validation_data=(Reshaped_X_test, to_categorical(Y_test, num_classes=256)),
                        batch_size=batch_size, verbose=1, epochs=epochs, callbacks=callbacks)
    return history

In [6]:

#region Model Configuration
MODEL_CONFIG = {
    'epochs': 100,
    'batch_size': 512,
    'input_size': 500,
    'learning_rate': 5e-3,
    'attack_traces': 5000,
    'model_save_path': './Model/TK1=0xdf.h5'
}
#endregion

#region Data Preparation
# Load dataset with proper error handling

(x_train, y_train, x_test, y_test, 
     p_train, p_test) = load_CW_Source(in_file='./DataSet/TK1=0xdf,TK2=0xab/',sec=18000)


# Preprocess targets
y_train = y_train[:]
y_test = y_test[:]

# Shuffle training data
x_train, y_train = shuffle(x_train, y_train)

# Standardization and Normalization
scaler = preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train).astype('float32')
x_test = scaler.transform(x_test).astype('float32')

minmax_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
x_train = minmax_scaler.fit_transform(x_train)
x_test = minmax_scaler.transform(x_test)
#endregion


#Model Initialization
try:
    model = cnn_classifier(
        input_size=MODEL_CONFIG['input_size'],
        learning_rate=MODEL_CONFIG['learning_rate']
    )
    model.summary()
except Exception as e:
    print(f"Model creation failed: {str(e)}")
    exit(1)

#region Training Execution
print("\n############### Starting Training #################\n")
try:
    training_history = train_model(
        x_train, y_train,
        x_test, y_test,
        model=model,
        save_file_name=MODEL_CONFIG['model_save_path'],
        epochs=MODEL_CONFIG['epochs'],
        batch_size=MODEL_CONFIG['batch_size'],
        max_lr=MODEL_CONFIG['learning_rate']
    )
except Exception as e:
    print(f"Training failed: {str(e)}")
    exit(1)

print("\n############### Training Completed #################\n")
#endregion

Model: "cnn_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500, 1)]          0         
_________________________________________________________________
block1_conv1 (Conv1D)        (None, 500, 4)            8         
_________________________________________________________________
batch_normalization (BatchNo (None, 500, 4)            16        
_________________________________________________________________
block1_pool (AveragePooling1 (None, 250, 4)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1000)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 16)                16016     
_________________________________________________________________
fc2 (Dense)                  (None, 16)             